In [95]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [96]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [364]:
doc = df_add[0]

In [365]:
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

#구두점 제거
doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()
print(len(doc1))

#숫자 제거
doc2 = "".join([i for i in doc1 if not i.isdigit()])
print(len(doc2))

#월 제거
month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
        'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
         'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   
    
doc3 = " ".join([i for i in doc2.split() if i not in month])
print(len(doc3))
print(doc3)

3882
3689
3429
Indexation of customs duty rates on exciseequivalent goods on August australian notice no indexation of duty rates on exciseequivalent goods on this notice sets out the new duty rates for certain exciseequivalent goods including spirits beers and fuel products operative from section of the tariff act customs tariff act provides for duty rates for certain exciseequivalent goods to be indexed biannually in and to the consumer price index cpi the new rates are determined by the application of an indexation factor this indexation factor is calculated by dividing the most recent or quarter cpi number by the previous highest or quarter cpi number occurring after the quarter the quarter cpi figure will be used to determine the indexation factor for the duty rate increase applied on as it is higher than the previously used quarter cpi figure refer to acn on the australian bureau of statistics released the quarter cpi figure the figures used to calculate the indexation factor for

In [366]:
n_gram_range = (2,2)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc3])
candidates = count.get_feature_names_out()

print('bigram 개수 :',len(candidates))
print('bigram 출력 :',candidates[:5])

bigram 개수 : 246
bigram 출력 : ['aac including' 'abf arrange' 'acn australian' 'act australian'
 'act customs']


In [367]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc3]) #전체 문서
candidate_embeddings = model.encode(candidates)#추출한 bigram

In [368]:
#문서와 가장 유사한 키워드 추출
doc_bigram_keywords = []

top_n = 20
distances = cosine_similarity(doc_embedding, candidate_embeddings)
doc_bigram_keyword = [candidates[index] for index in distances.argsort()[0][-top_n:]]
doc_bigram_keywords.append(doc_bigram_keyword)


print(len(doc_bigram_keywords))
print(doc_bigram_keywords)

1
[['australian notice', 'indexed biannually', 'schedule indonesian', 'acn australian', 'atogovaualcoholexciserates atogovaufuelexciserates', 'performance australian', 'spirits beers', 'policy australian', 'aseanaustralianew zealand', 'gazette australian', 'quarter indexation', 'goods australia', 'atogovaufuelexciserates fuel', 'outlined australian', 'tariff subheadings', 'tariff amendment', 'august australian', 'australian taxation', 'schedule aseanaustralianew', 'revised tariff']]


In [369]:
#문서와 가장 유사한 키워드 벡터화
doc_bigram_embedding = []
for keyword in doc_bigram_keywords:
    doc_bigram_embedding.append(model.encode(keyword))

In [370]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

102


In [371]:
#문서 키워드(doc_bigram_embedding)와 키워드 100(keyword_embedding) 유사도 비교 
bigram_key_result = []
keyword_key_result = []
cosine_distances = []

for index, bigram in enumerate(doc_bigram_embedding):
    bigram_result = []
    key_result=[]
    cosine_distance = []
    for i in range(len(bigram)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([bigram[i]],[keyword_embedding[j]])
            if distances[0][0] > 0.75:
                bigram_result.append(doc_bigram_keywords[index][i])
                key_result.append(keyword[j])
                cosine_distance.append(distances)

    bigram_key_result.append(bigram_result)
    keyword_key_result.append(key_result)
    cosine_distances.append(cosine_distance)

In [372]:
df_check = pd.DataFrame()

print(bigram_key_result)
print(keyword_key_result)

for i in range(len(bigram_key_result)):
    bigram = pd.Series(bigram_key_result[i])
    keyword = pd.Series(keyword_key_result[i])
    distance = pd.Series(cosine_distances[i])
    
    df_check['bigram'] = bigram
    df_check['keyword'] = keyword
    df_check['distance'] = distance

df_check = df_check.sort_values(by="distance", ascending=False)
df_check

[['spirits beers']]
[['alcohol']]


,bigram,keyword,distance
0,spirits beers,alcohol,[[0.7743218]]
